In [1]:
library(dlnm)
library(mgcv)
library(data.table)
library(plotly)
library(ggplot2)
library(splines)
library(parallel)
library(doParallel)
library(foreach)
library(coda)
library(lubridate)
library(xts)
library(timetk)
library(forecast)
library(gridExtra)
library(tidyr)
library(dplyr)
library(Hmisc)
library(xtable)
library(MuMIn)
library(dlnm) ; library(splines) ; library(MASS) ; library(tsModel)

This is dlnm 2.4.7. For details: help(dlnm) and vignette('dlnmOverview').

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.

Loading required package: ggplot2


Attaching package: 'plotly'


The following object is masked from 'package:ggplot2':

    last_plot


The following object is masked from 'package:stats':

    filter


The following object is masked from 'package:graphics':

    layout


Loading required package: foreach

Loading required package: iterators


Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



Attaching package: 'xts'


The following objects are masked fro

# AT01 

In [2]:
################################################################################
# READ DATA
################################################################################
data <- read.csv("modelling_data/monthly_data.csv")
dlnm_selection <- head(read.csv("dlnm_tuning/AT01_dlnm_model_selection.csv"),24)

In [3]:
dlnm_selection

,model,aic,deviance,year_df,has_holiday,spline_fun,df,data_split
,<chr>,<dbl>,<dbl>,<int>,<lgl>,<chr>,<int>,<chr>
1,ns_3_2_2_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,2,max
2,ns_3_2_2_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,2,max
3,ns_3_3_3_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,3,max
4,ns_3_3_3_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,3,max
5,ns_3_4_4_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,4,max
6,ns_3_4_4_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,4,max
7,ns_3_5_5_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,5,max
8,ns_3_5_5_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,5,max
9,ns_3_6_6_hum_temp_so2_no2_without_holiday,860.6577,116.8529,3,FALSE,ns,6,max


In [4]:

################################################################################
# DATA VARIABLES SPLIT
################################################################################
# # MAX
# ################################################################################
data <- data %>% dplyr:::select(
                                at_code,
                                events,
                                temp = max_temp,
                                hum = max_hum,
                                is_holiday,
                                month,
                                year,
                                pm10 = max_PM10,
                                o3 = max_O3,
                                no2 = max_NO2,
                                so2 = max_SO2,
                                co = max_CO
                                )

#January, February, November, and December <– Paper 2019

# Function to categorize each month into a season
categorize_season <- function(month) {
  if (month %in% c(1, 2, 11, 12)) {
    return("Winter")
  } else {
    return("Summer")
  }
}

# Add a new column "season" to the environmeAMIet based on the month
data$season <- sapply(data$month, categorize_season)

# ################################################################################
# AT CODE
# ################################################################################
data = data[(data$at_code=="AT01"),]
#data$date <- as.Date(data$date)
# Simulate a date column based on month and year
#data$date <- as.Date(paste(data$year, data$month, "01", sep="-"))

In [5]:
library(dlnm)
library(splines)
library(progress) 

# Possible degrees of freedom for argvar and arglag
df_options <- c(2, 3)

# Dataframe to store results
results <- data.frame(model = character(), variables = character(), df_pm10 = character(), df_hum = character(), 
                      df_no2 = character(), df_temp = character(), df_so2 = character(), df_co = character(), 
                      is_holiday = logical(), residual_dispersion = numeric(), AIC = numeric(), stringsAsFactors = FALSE)

# Define variable combinations for modeling
variable_sets <- list(
  c("hum"),
  c("temp"),
  c("pm10"),
  c("so2"),
  c("hum", "temp"),
  c("hum", "o3"),
  c("hum", "temp", "pm10"),
  c("hum", "temp", "no2"),
  c("hum", "temp", "so2"),
  c("hum", "temp", "co"),
  c("hum", "temp", "pm10", "o3"),
  c("hum", "temp", "pm10", "no2"),
  c("hum", "temp", "so2", "no2"),
  c("hum", "temp", "pm10", "o3", "no2"),
  c("hum", "temp", "pm10", "so2", "no2"),
  c("hum", "temp", "pm10", "o3", "no2", "so2"),
  c("hum", "temp", "pm10", "o3", "no2", "so2", "co")
)

# Generate all combinations of df settings
df_combinations <- expand.grid(df_pm10_var=df_options, df_pm10_lag=df_options, 
                               df_hum_var=df_options, df_hum_lag=df_options, 
                               df_no2_var=df_options, df_no2_lag=df_options, 
                               df_temp_var=df_options, df_temp_lag=df_options, 
                               df_so2_var=df_options, df_so2_lag=df_options, 
                               df_co_var=df_options, df_co_lag=df_options)

# Calculate the total number of iterations
total_iterations <- nrow(df_combinations) * length(variable_sets) * 2  # 2 for holiday inclusion/exclusion

# Initialize tqdm progress bar
pb <- txtProgressBar(min = 0,      # Minimum value of the progress bar
                     max = total_iterations, # Maximum value of the progress bar
                     style = 3,    # Progress bar style (also available style = 1 and style = 2)
                     width = 50,   # Progress bar width. Defaults to getOption("width")
                     char = "=")   # Character used to create the bar

# Loop over each combination of df settings, variable sets, and holiday inclusion
for (i in 1:nrow(df_combinations)) {
  df_comb <- df_combinations[i,]
  
  # Create crossbasis functions with current df settings
  cb_pm10 <- crossbasis(data$pm10, lag=12, 
                        argvar=list(fun="ns", df=df_comb$df_pm10_var), 
                        arglag=list(fun="ns", df=df_comb$df_pm10_lag))
  
  cb_hum <- crossbasis(data$hum, lag=12, 
                       argvar=list(fun="ns", df=df_comb$df_hum_var), 
                       arglag=list(fun="ns", df=df_comb$df_hum_lag))
  
  cb_no2 <- crossbasis(data$no2, lag=12, 
                       argvar=list(fun="ns", df=df_comb$df_no2_var), 
                       arglag=list(fun="ns", df=df_comb$df_no2_lag))
  
  cb_temp <- crossbasis(data$temp, lag=12, 
                        argvar=list(fun="ns", df=df_comb$df_temp_var), 
                        arglag=list(fun="ns", df=df_comb$df_temp_lag))
  
  cb_so2 <- crossbasis(data$so2, lag=12, 
                       argvar=list(fun="ns", df=df_comb$df_so2_var), 
                       arglag=list(fun="ns", df=df_comb$df_so2_lag))
  
  cb_co <- crossbasis(data$co, lag=12, 
                      argvar=list(fun="ns", df=df_comb$df_co_var), 
                      arglag=list(fun="ns", df=df_comb$df_co_lag))
  
  for (vars in variable_sets) {
    for (holiday in c(TRUE, FALSE)) {
      # Build the formula dynamically
      formula_vars <- paste(vars, collapse = " + ")
      if (holiday) {
        formula_vars <- paste(formula_vars, "+ is_holiday")
      }
      formula <- as.formula(paste("events ~", formula_vars, "+ ns(year, df=3) + factor(season)"))
      
      # Fit the model
      model <- glm(formula, data = data, family = poisson)
      
      # Calculate residual dispersion
      residual_deviance <- model$deviance
      
      # Calculate AIC
      aic <- AIC(model)
      
      # Store the results
      results <- rbind(results, data.frame(model = paste("Model", i), 
                                           variables = formula_vars, 
                                           df_pm10 = paste(df_comb$df_pm10_var, df_comb$df_pm10_lag, sep = ","), 
                                           df_hum = paste(df_comb$df_hum_var, df_comb$df_hum_lag, sep = ","), 
                                           df_no2 = paste(df_comb$df_no2_var, df_comb$df_no2_lag, sep = ","), 
                                           df_temp = paste(df_comb$df_temp_var, df_comb$df_temp_lag, sep = ","), 
                                           df_so2 = paste(df_comb$df_so2_var, df_comb$df_so2_lag, sep = ","), 
                                           df_co = paste(df_comb$df_co_var, df_comb$df_co_lag, sep = ","), 
                                           is_holiday = holiday, 
                                           residual_deviance = residual_deviance, 
                                           AIC = aic))
      # Update progress bar - sets the progress bar to the current state
      setTxtProgressBar(pb, i)  # Update the progress bar
    }
  }
}

# Close the progress bar when done
close(progress_bar)

  |                                                  |   1%

In [ ]:
# Just append the previously found results
results['data_split'] <- 'max'
results['year_df'] <- 3
results <- results %>% arrange(AIC, residual_deviance)

# SAVE
write.csv(results, "dlnm_tuning/AT01_best_model.csv", row.names = FALSE)

# BEST MODEL

In [ ]:
# ################################################################################
# CROSS BASIS FUNCTIONS WITH GIVEN DF AND VARIABLES
# ################################################################################
# Create crossbasis for mean_PM10 and min_hum and ozone
cb_pm10 <- crossbasis(data$pm10, lag=12, 
                      argvar=list(fun="ns", df=3), 
                      arglag=list(fun="ns", df=3))

cb_hum <- crossbasis(data$hum, lag=12, 
                         argvar=list(fun="ns", df=3), 
                         arglag=list(fun="ns", df=3))

cb_no2 <- crossbasis(data$no2, lag=12, 
                         argvar=list(fun="ns", df=2), 
                         arglag=list(fun="ns", df=2))

cb_temp <- crossbasis(data$temp, lag=12, 
                    argvar=list(fun="ns", df=3), 
                    arglag=list(fun="ns", df=3))

cb_so2 <- crossbasis(data$so2, lag=12, 
                    argvar=list(fun="ns", df=2), 
                    arglag=list(fun="ns", df=2))

cb_co <- crossbasis(data$co, lag=12, 
                    argvar=list(fun="ns", df=2), 
                    arglag=list(fun="ns", df=2))

In [ ]:
# ################################################################################
# MODEL FITTING
# ################################################################################
# Fit DLNM including min_hum
model <- glm(events ~ cb_pm10 + cb_hum + cb_no2 + cb_temp + ns(year, df=3) + factor(season), 
             data = data, family = quasipoisson)
# ################################################################################
# SUMMARY
# ################################################################################
# Summary of the model
summary(model)
# ################################################################################
# DIAGNOSTICS
# ################################################################################
plot(model$residuals, main="Residuals of the Model", ylab="Residuals", xlab="Index")
abline(h=0, col="red")

qqnorm(model$residuals)
qqline(model$residuals, col="red")

acf(model$residuals)
# ################################################################################
# RESULTS
# ################################################################################
par(mfrow = c(2, 2))

# Create crosspred object for visualization of temp
pred_temp <- crosspred(cb_temp, model, at=seq(min(data$temp), max(data$temp), length=100))
# Plot overall cumulative effect
plot(pred_temp, "overall", xlab="Temp", ylab="Relative Risk", main="Overall Cumulative Effect of Temp")

# Create crosspred object for visualization of hum
pred_hum <- crosspred(cb_hum, model, at=seq(min(data$hum), max(data$hum), length=100))
# Plot overall cumulative effect
plot(pred_hum, "overall", xlab="Hum", ylab="Relative Risk", main="Overall Cumulative Effect of Hum")

# Create crosspred object for visualization of no2
pred_no2 <- crosspred(cb_no2, model, at=seq(min(data$no2), max(data$no2), length=100))
# Plot overall cumulative effect
plot(pred_no2, "overall", xlab="NO2", ylab="Relative Risk", main="Overall Cumulative Effect of NO2")

# Create crosspred object for visualization of pm10
pred_pm10 <- crosspred(cb_pm10, model, at=seq(min(data$pm10), max(data$pm10), length=100))
# Plot overall cumulative effect
plot(pred_pm10, "overall", xlab="PM10", ylab="Relative Risk", main="Overall Cumulative Effect of PM10")

# ################################################################################
# SLICES
# ################################################################################
pdf("plots/DLNM/overall_cumulative_effect.pdf")
par(mfrow = c(2, 2))
plot(pred_temp, "slices", xlab="Temperature", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_hum, "slices", xlab="Humidity", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_no2, "slices", xlab="NO2", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_pm10, "slices", xlab="PM10", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
# Close the PDF device
dev.off()
# ################################################################################
# CONTOURS
# ################################################################################
par(mfrow = c(1, 1))
# 3D, contour, and curve plots for temperature
#plot(pred_temp, xlab="Temperature", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
pdf("plots/DLNM/contour_temp.pdf")
plot(pred_temp, "contour", key.title=title("events"), plot.title=title("", xlab ="Temperature", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
dev.off()

# 3D, contour, and curve plots for humidity
#plot(pred_hum, xlab="Humidity", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
pdf("plots/DLNM/contou_hum.pdf")
plot(pred_hum, "contour", key.title=title("events"), plot.title=title("", xlab ="Humidity", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
dev.off()

# 3D, contour, and curve plots for no2
#plot(pred_no2, xlab="NO2", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
pdf("plots/DLNM/contou_no2.pdf")
plot(pred_no2, "contour", key.title=title("events"), plot.title=title("", xlab ="NO2", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
dev.off()

# 3D, contour, and curve plots for pm10
#plot(pred_pm10, xlab="PM10", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
pdf("plots/DLNM/contou_pm10.pdf")
plot(pred_pm10, "contour", key.title=title("events"), plot.title=title("", xlab ="pm10", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
dev.off()



In [ ]:
################################################################################
# READ DATA
################################################################################
data <- read.csv("modelling_data/monthly_data.csv")

data <- data %>% dplyr:::select(
                                at_code,
                                events,
                                temp = max_temp,
                                hum = min_hum,
                                is_holiday,
                                month,
                                year,
                                pm10 = mean_PM10,
                                o3 = mean_O3,
                                no2 = mean_NO2,
                                so2 = mean_SO2,
                                co = mean_CO
                                )

#January, February, November, and December <– Paper 2019

# Function to categorize each month into a season
categorize_season <- function(month) {
  if (month %in% c(1, 2, 11, 12)) {
    return("Winter")
  } else {
    return("Summer")
  }
}

# Add a new column "season" to the environmeAMIet based on the month
data$season <- sapply(data$month, categorize_season)
# ################################################################################
# AT CODE
# ################################################################################
data = data[(data$at_code=="AT01"),]
#data$date <- as.Date(data$date)
# Simulate a date column based on month and year
#data$date <- as.Date(paste(data$year, data$month, "01", sep="-"))

# ################################################################################
# CROSS BASIS FUNCTIONS
# ################################################################################
# Create crossbasis for mean_PM10 and min_hum and ozone
cb_pm10 <- crossbasis(data$pm10, lag=12, 
                      argvar=list(fun="ns", df=3), 
                      arglag=list(fun="ns", df=3))

cb_hum <- crossbasis(data$hum, lag=12, 
                         argvar=list(fun="ns", df=3), 
                         arglag=list(fun="ns", df=3))

cb_no2 <- crossbasis(data$no2, lag=12, 
                         argvar=list(fun="ns", df=2), 
                         arglag=list(fun="ns", df=2))

cb_temp <- crossbasis(data$temp, lag=12, 
                    argvar=list(fun="ns", df=3), 
                    arglag=list(fun="ns", df=3))

cb_so2 <- crossbasis(data$so2, lag=12, 
                    argvar=list(fun="ns", df=2), 
                    arglag=list(fun="ns", df=2))

cb_co <- crossbasis(data$co, lag=12, 
                    argvar=list(fun="ns", df=2), 
                    arglag=list(fun="ns", df=2))
# ################################################################################
# MODEL FITTING
# ################################################################################
# Fit DLNM including min_hum
model <- glm(events ~ cb_pm10 + cb_hum + cb_no2 + cb_temp + ns(year, df=9) + factor(season), 
             data = data, na.action=na.exclude, family = poisson)
# ################################################################################
# SUMMARY
# ################################################################################
# Summary of the model
summary(model)
# ################################################################################
# DIAGNOSTICS
# ################################################################################
par(mfrow = c(2, 2))
plot(model$residuals, main="Residuals of the Model", ylab="Residuals", xlab="Index")
abline(h=0, col="red")

dispersion <- sum(residuals(model, type="pearson")^2) / model$df.residual
print(paste("Dispersion parameter:", dispersion))

qqnorm(model$residuals)
qqline(model$residuals, col="red")

acf(model$residuals)
pacf(model$residuals)


# Compute deviance residuals
deviance_residuals <- residuals(model, type = "deviance")

# Create lagged residuals at lag 1
lagged_residuals <- Lag(deviance_residuals, 6)
# Update the model by adding lagged residuals
model <- update(model, . ~ . + lagged_residuals)

par(mfrow = c(2, 2))

plot(model$residuals, main="Residuals of the Model", ylab="Residuals", xlab="Index")
abline(h=0, col="red")

dispersion <- sum(residuals(model, type="pearson")^2) / model$df.residual
print(paste("Dispersion parameter:", dispersion))

qqnorm(model$residuals)
qqline(model$residuals, col="red")
acf(model$residuals)
pacf(residuals(model,type="deviance"),na.action=na.omit,main="Autocorrelation (adjusted model)",xlim=c(0,20))

par(mfrow = c(1, 1))
# Compute deviance residuals
deviance_residuals <- residuals(model, type = "deviance")


# ################################################################################
# RESULTS
# ################################################################################
par(mfrow = c(2, 2))

# Create crosspred object for visualization of temp
pred_temp <- crosspred(cb_temp, model, at=seq(min(data$temp), max(data$temp), length=100))
# Plot overall cumulative effect
plot(pred_temp, "overall", xlab="Temp", ylab="Relative Risk", main="Overall Cumulative Effect of Temp")

# Create crosspred object for visualization of hum
pred_hum <- crosspred(cb_hum, model, at=seq(min(data$hum), max(data$hum), length=100))
# Plot overall cumulative effect
plot(pred_hum, "overall", xlab="Hum", ylab="Relative Risk", main="Overall Cumulative Effect of Hum")

# Create crosspred object for visualization of no2
pred_no2 <- crosspred(cb_no2, model, at=seq(min(data$no2), max(data$no2), length=100))
# Plot overall cumulative effect
plot(pred_no2, "overall", xlab="NO2", ylab="Relative Risk", main="Overall Cumulative Effect of NO2")

# Create crosspred object for visualization of pm10
pred_pm10 <- crosspred(cb_pm10, model, at=seq(min(data$pm10), max(data$pm10), length=100))
# Plot overall cumulative effect
plot(pred_pm10, "overall", xlab="PM10", ylab="Relative Risk", main="Overall Cumulative Effect of PM10")

# ################################################################################
# SLICES
# ################################################################################
#pdf("plots/DLNM/overall_cumulative_effect.pdf")
par(mfrow = c(2, 2))
plot(pred_temp, "slices", xlab="Temperature", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_hum, "slices", xlab="Humidity", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_no2, "slices", xlab="NO2", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
plot(pred_pm10, "slices", xlab="PM10", lag=c(2), col="orange2", ci.arg=list(col=topo.colors(70, alpha = 1)), ci.level=0.95, ci='b', lwd=4.5, ylab="RR of events of AMI", cex.lab=1.1, cex.axis=1.1,main="")
# Close the PDF device
#dev.off()
# ################################################################################
# CONTOURS
# ################################################################################
par(mfrow = c(1, 1))
# 3D, contour, and curve plots for temperature
#plot(pred_temp, xlab="Temperature", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
#pdf("plots/DLNM/contour_temp.pdf")
plot(pred_temp, "contour", key.title=title("events"), plot.title=title("", xlab ="Temperature", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
#dev.off()

# 3D, contour, and curve plots for humidity
#plot(pred_hum, xlab="Humidity", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
#pdf("plots/DLNM/contou_hum.pdf")
plot(pred_hum, "contour", key.title=title("events"), plot.title=title("", xlab ="Humidity", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
#dev.off()

# 3D, contour, and curve plots for no2
#plot(pred_no2, xlab="NO2", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
#pdf("plots/DLNM/contou_no2.pdf")
plot(pred_no2, "contour", key.title=title("events"), plot.title=title("", xlab ="NO2", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
#dev.off()

# 3D, contour, and curve plots for pm10
#plot(pred_pm10, xlab="PM10", ylab="Lag (days)", zlab="RR of events of AMI", theta=150, phi=5, lphi=100, cex.lab=1.1, cex.axis=1.1, col="gray80",main="")
#pdf("plots/DLNM/contou_pm10.pdf")
plot(pred_pm10, "contour", key.title=title("events"), plot.title=title("", xlab ="pm10", ylab = "Lag (days)", cex.lab=1.1, cex.axis=1.1))
#dev.off()
# ################################################################################
# LAGS
# ################################################################################
# Load necessary libraries
library(ggplot2)
library(gridExtra)
library(dlnm)  # Assuming you are using the dlnm package for crosspred

# Function to extract a specific row by value and create a lag plot data frame
create_lag_plot_by_value <- function(pred_obj, value) {
  row_index <- which(abs(pred_obj$predvar - value) == min(abs(pred_obj$predvar - value)))
  data.frame(
    index = 1:length(pred_obj$matRRfit[row_index, ]),
    RRfit = as.numeric(pred_obj$matRRfit[row_index, ]),
    RRlow = as.numeric(pred_obj$matRRlow[row_index, ]),
    RRhigh = as.numeric(pred_obj$matRRhigh[row_index, ])
  )
}

# Specify the value you are interested in
temp_value <- 13.1948799826219
hum_value <- 19.9448508069198
no2_value <- 25.0244971454727
pm10_value <- 33.4198743176959

# Create lag plots for each crosspred object
lag_plot_temp <- create_lag_plot_by_value(pred_temp, temp_value)
lag_plot_hum <- create_lag_plot_by_value(pred_hum, chosen_value)
lag_plot_no2 <- create_lag_plot_by_value(pred_no2, chosen_value)
lag_plot_pm10 <- create_lag_plot_by_value(pred_pm10, chosen_value)

# Function to plot data
plot_lag <- function(data, xlab, ylab, main) {
  ggplot(data, aes(x = index)) +
    geom_line(aes(y = RRfit), color = "orange2", size = 1.5) +
    geom_line(aes(y = 1), color = "black", size = 1, alpha = 0.5) +
    geom_ribbon(aes(ymin = RRlow, ymax = RRhigh), fill = "lightblue", alpha = 0.5) +
    labs(title = main, x = xlab, y = ylab) +
    theme_minimal() +
    theme(
      text = element_text(size = 10),
      axis.title = element_text(size = 12),
      axis.text = element_text(size = 10)
    )
}

# Create individual plots
plot_temp <- plot_lag(lag_plot_temp, "Lag (month)", "RR of events of AMI", "Temperature")
plot_hum <- plot_lag(lag_plot_hum, "Lag (month)", "RR of events of AMI", "Humidity")
plot_no2 <- plot_lag(lag_plot_no2, "Lag (month)", "RR of events of AMI", "NO2")
plot_pm10 <- plot_lag(lag_plot_pm10, "Lag (month)", "RR of events of AMI", "PM10")

# Arrange the plots in a 2x2 grid
grid.arrange(plot_temp, plot_hum, plot_no2, plot_pm10, nrow = 2, ncol = 2)


# SCRIPTS

In [ ]:
################################################################################
# DATA VARIABLES SPLIT
################################################################################
# MIN
################################################################################
# data <- data %>% dplyr:::select(
#                                 at_code,
#                                 events,
#                                 temp = min_temp,
#                                 hum = min_hum,
#                                 is_holiday,
#                                 month,
#                                 year,
#                                 pm10 = min_PM10,
#                                 o3 = min_O3,
#                                 no2 = min_NO2,
#                                 so2 = min_SO2,
#                                 co = min_CO
#                                 )

# ################################################################################
# # MAX
# ################################################################################
# data <- data %>% dplyr:::select(
#                                 at_code,
#                                 events,
#                                 temp = max_temp,
#                                 hum = max_hum,
#                                 is_holiday,
#                                 month,
#                                 year,
#                                 pm10 = max_PM10,
#                                 o3 = max_O3,
#                                 no2 = max_NO2,
#                                 so2 = max_SO2,
#                                 co = max_CO
#                                 )

################################################################################
# MEAN
# ################################################################################
#  data <- data %>% dplyr:::mutate(
#     mean_temp = (max_temp + min_temp) / 2,
#     mean_hum = (max_hum + min_hum) / 2
#           ) %>% dplyr:::select(
#                                 at_code,
#                                 events,
#                                 temp = mean_temp,
#                                 hum = mean_hum,
#                                 is_holiday,
#                                 month,
#                                 year,
#                                 pm10 = mean_PM10,
#                                 o3 = mean_O3,
#                                 no2 = mean_NO2,
#                                 so2 = mean_SO2,
#                                 co = mean_CO
#                                 )

# ################################################################################
# # MIX
# ################################################################################
# data <- data %>% dplyr:::select(
#                                 at_code,
#                                 events,
#                                 temp = max_temp,
#                                 hum = min_hum,
#                                 is_holiday,
#                                 month,
#                                 year,
#                                 pm10 = mean_PM10,
#                                 o3 = mean_O3,
#                                 no2 = mean_NO2,
#                                 so2 = mean_SO2,
#                                 co = mean_CO
#                                 )